# Import packages

In [1]:
import MetaTrader5 as mt
import pandas as pd
import ml_collections
import yaml

from datetime import datetime
import pytz
import sys

sys.path.insert(0, 'D:/Intraday_trading')
from src.support import *

import time
from IPython.display import clear_output

In [2]:
config_dir = 'D:/Intraday_trading/configs/config.yaml'
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))


In [3]:
acc_config = config.real_acc

# Login account

In [4]:
login_metatrader(acc_config = acc_config)

# Deals monitor 

In [ ]:
from_date=datetime(2024,1,1)
to_date=datetime.now()
# get deals for symbols whose names contain "XAUUSDm" within a specified interval
deals = mt.history_deals_get(from_date, to_date, group="*")

df_deals = pd.DataFrame(deals, columns = deals[0]._asdict().keys())

df_deals['time'] = pd.to_datetime(df_deals['time'], unit = 's')
df_deals['time_msc'] = pd.to_datetime(df_deals['time_msc'], unit = 'ms')

df_deals['VN_time'] = df_deals['time'] + pd.Timedelta('7 hours')
df_deals['VN_time_msc'] = df_deals['time_msc'] + pd.Timedelta('7 hours')


In [ ]:
ref_deals_entry = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_entry.csv', encoding_errors = 'ignore')
ref_deals_reason = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_reason.csv', encoding_errors = 'ignore')
ref_deals_type = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_type.csv', encoding_errors = 'ignore')


In [ ]:
df_deals = df_deals.merge(ref_deals_entry, how = 'left', left_on = 'type', right_on = ref_deals_entry.index)
df_deals = df_deals.merge(ref_deals_reason, how = 'left', left_on = 'type', right_on = ref_deals_reason.index)
df_deals = df_deals.merge(ref_deals_type, how = 'left', left_on = 'type', right_on = ref_deals_type.index)

In [ ]:
df_deals.tail(6)

# Orders monitor

In [ ]:
# from_date=datetime(2024,6,15)
# to_date=datetime.now()
# # get deals for symbols whose names contain "XAUUSDm" within a specified interval
# orders = mt.history_orders_get(from_date, to_date, group="*XAUUSDm*")

# df_orders = pd.DataFrame(orders, columns = orders[0]._asdict().keys())
# df_orders['time_setup'] = pd.to_datetime(df_orders['time_setup'], unit = 's')
# df_orders['VN_time'] = df_orders['time_setup'] + pd.Timedelta('7 hours')

In [ ]:
# df_orders

# Data

## OHLCV interval data

In [ ]:
# Compute now date
from_date = datetime(2001, 1, 1)
to_date = datetime(2024, 4, 11)


rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M5, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M15, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_H1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_D1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])

## Ticks data

In [ ]:
# https://thiagomarzagao.com/2021/05/12/mt5/

In [ ]:
# Compute now date
from_date = datetime(2020, 4, 10)
to_date = datetime(2024, 4, 11)

# Extract n Ticks before now
ticks = mt.copy_ticks_range("EURUSDm", from_date, to_date, mt.COPY_TICKS_ALL)

# Transform Tuple into a DataFrame
df_ticks = pd.DataFrame(ticks)

# Convert number format of the date into date format
df_ticks["time"] = pd.to_datetime(df_ticks["time"], unit="s")


df_ticks